In [1]:
import logging
import time
import akshare as ak
import sqlalchemy as db  # Version Check
import pandas as pd
import logging
import sqlalchemy as db 
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import MetaData, Table, Sequence, Column, Float, Integer, String
from sqlalchemy.sql import select


In [2]:
def str2str(s, strp, strf):
    return time.strftime(
        strf, 
        time.strptime(
            s, 
            strp)
    )


In [3]:
connection_uri = "mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}".format(**{
    "user": "root",
    "password": "mysql123456",
    "host": "localhost",
    "port": "3306",
    "dbname": "xinguan"
})
engine = db.create_engine(connection_uri)


In [4]:
metadata = MetaData()

t_daily = Table(
    'csse_daily',
    metadata,
    Column('us_only_county_code_fips', Integer),
    Column('us_only_county_name', String(64)),
    Column('higher', String(64)),
    Column('lower', String(64)),
    Column('last_update', String(64)),
    Column('last_update_unify', String(64)),
    Column('report_day', String(64)),
    Column('latitude', Float),
    Column('longitude', Float),
    Column('confirmed', Integer),
    Column('dead', Integer),
    Column('recovered', Integer),
    Column('active', Integer),
    Column('district', String(64)),
)

t_daily.drop(engine, checkfirst=True)
t_daily.create(engine)



In [5]:
retry = 100
report_day = "2020-03-01"
for j in range(retry):
    try:
        df0301 = ak.covid_19_csse_daily(date=report_day)
        logging.info("成功WEB：{0}".format(report_day))
        print("成功WEB：{0}".format(report_day))
        time.sleep(0.1)
        break
    except Exception as e:
        if j == retry - 1:
            logging.info(e)
            logging.info("失败WEB：{0}".format(report_day))
            print("失败WEB：{0}".format(report_day))
        logging.info(e)
        logging.info("重试WEB：{0}".format(report_day))
        print("重试WEB：{0}".format(report_day))


成功WEB：2020-03-01


In [6]:
df0301.rename(columns={
    'Province/State':'higher',
    'Country/Region':'lower',
    'Last Update':'last_update',
    'Latitude':'latitude',
    'Longitude':'longitude',
    'Confirmed':'confirmed',
    'Deaths':'dead',
    'Recovered':'recovered'
}, inplace=True)


In [7]:
df0301['last_update_unify'] = df0301['last_update'].map(lambda x:str2str(x, '%Y-%m-%dT%H:%M:%S', '%Y-%m-%dT%H:%M:%S'))     
# df0301['report_day'] = df0301['last_update'].map(lambda x:str2str(x, '%Y-%m-%dT%H:%M:%S', '%Y-%m-%d')) 
df0301['report_day'] = report_day

df0301.fillna(value={
    'higher':'null',
    'lower':'null',
    'confirmed':0,
    'dead':0,
    'recovered':0,
    
}, inplace=True)
df0301

,higher,lower,last_update,confirmed,dead,recovered,latitude,longitude,last_update_unify,report_day
0,Hubei,Mainland China,2020-03-01T10:13:19,66907,2761,31536,30.9756,112.2707,2020-03-01T10:13:19,2020-03-01
1,null,South Korea,2020-03-01T23:43:03,3736,17,30,36.0000,128.0000,2020-03-01T23:43:03,2020-03-01
2,null,Italy,2020-03-01T23:23:02,1694,34,83,43.0000,12.0000,2020-03-01T23:23:02,2020-03-01
3,Guangdong,Mainland China,2020-03-01T14:13:18,1349,7,1016,23.3417,113.4244,2020-03-01T14:13:18,2020-03-01
4,Henan,Mainland China,2020-03-01T14:13:18,1272,22,1198,33.8820,113.6140,2020-03-01T14:13:18,2020-03-01
...,...,...,...,...,...,...,...,...,...,...
120,"Tempe, AZ",US,2020-02-25T21:23:03,1,0,1,33.4255,-111.9400,2020-02-25T21:23:03,2020-03-01
121,From Diamond Princess,Australia,2020-02-29T02:03:10,0,0,0,35.4437,139.6380,2020-02-29T02:03:10,2020-03-01
122,"Lackland, TX (From Diamond Princess)",US,2020-02-24T23:33:02,0,0,0,29.3829,-98.6134,2020-02-24T23:33:02,2020-03-01
123,"Omaha, NE (From Diamond Princess)",US,2020-02-24T23:33:02,0,0,0,41.2545,-95.9758,2020-02-24T23:33:02,2020-03-01


In [8]:
for j in range(retry):
    try:
        df0301.to_sql(
            t_daily.name,
            con=engine,
            if_exists='append',
            index=False)
        logging.info("成功DB：{0}".format(report_day))
        print("成功DB：{0}".format(report_day))
        time.sleep(0.1)
        break
    except Exception as e:
        if j == retry - 1:
            logging.info("失败DB：{0}".format(report_day))
            print("失败DB：{0}".format(report_day))
        logging.info("重试DB：{0}".format(report_day))
        print("重试DB：{0}".format(report_day))
        logging.info(e)


成功DB：2020-03-01


* `>= 01-22-2020.csv`
* `>= 03-01-2020.csv`
* `>= 03-22-2020.csv`

***


* FIPS: US only. FederalInformationProcessingStandards. Code that uniquely identifies counties within the USA.
* Admin2: US only. County name. 
* Province_State: Province, state or dependency name. 
    * Province/State: 
        * China - province name; 
        * US/Canada/Australia/ - city name, state/province name; 
        * Others - name of the event (e.g., "Diamond Princess" cruise ship); 
        * other countries - blank.
* Country_Region: 
    * Country, region or sovereignty name. The names of locations included on the Website correspond with the official designations used by the U.S. Department of State.
* Last Update: MM/DD/YYYY HH:mm:ss (24 hour format, in UTC).
* Lat and Long_: Dot locations on the dashboard. All points (except for Australia) shown on the map are based on geographic centroids, and are not representative of a specific address, building or any location at a spatial scale finer than a province/state. Australian dots are located at the centroid of the largest city in each state.
* Confirmed: Confirmed cases include `presumptive positive` cases and `probable` cases, in accordance with CDC guidelines as of April 14.
* Deaths: Death totals in the US include confirmed and probable, in accordance with CDC guidelines as of April 14.
* Recovered: Recovered cases outside China are estimates based on local media reports, and state and local reporting when available, and therefore may be substantially lower than the true number. US state-level recovered cases are from COVID Tracking Project.
* Active: Active cases = total confirmed - total recovered - total deaths.
* Incidence_Rate: Admin2 + Province_State + Country_Region.
* Case-Fatality Ratio (%): = confirmed cases per 100,000 persons.
* US Testing Rate: = total test results per 100,000 persons. The "total test results" is equal to "Total test results (Positive + Negative)" from COVID Tracking Project.
* US Hospitalization Rate (%): = Total number hospitalized / Number confirmed cases. The "Total number hospitalized" is the "Hospitalized – Cumulative" count from COVID Tracking Project. The "hospitalization rate" and "hospitalized - Cumulative" data is only presented for those states which provide cumulative hospital data.

***

Update frequency:
* Files on and after April 23, once per day between 03:30 and 04:00 UTC.
* Files from February 2 to April 22: once per day around 23:59 UTC.
* Files on and before February 1: the last updated files before 23:59 UTC. Sources: archived_data and dashboard.